In [1]:
import pandas as pd

data_path = "/global/scratch/users/toniooppi/nuclear_data/cmsr_dataset_material_movement.h5"
df_base = pd.read_hdf(data_path)

In [2]:
df = df_base.copy()

In [3]:
df

,IE,BU,CT,FMI,fuel_H1,fuel_H2,fuel_H3,fuel_He3,fuel_He4,fuel_Li6,...,flow_Sr91,flow_Sr89,flow_Tc103,flow_Mo99,flow_Pr147,flow_Y95,flow_Sb132,flow_Sb129,AN,SF
0,10.0,0.0000,0.0,0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,7.310000e-20,4.470000e-21,2.110000e-20,1.090000e-20,9.150000e-18,3.000000e-19,3.680000e-19,1.980000e-20,0.8384,0.02793
1,10.0,0.0000,0.0,1,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,8.105500e-19,4.956441e-20,2.339618e-19,1.208618e-19,1.014574e-16,3.326471e-18,4.080471e-18,2.195471e-19,0.8384,0.02793
2,10.0,0.0000,0.0,2,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,1.548000e-18,9.465882e-20,4.468235e-19,2.308235e-19,1.937647e-16,6.352941e-18,7.792941e-18,4.192941e-19,0.8384,0.02793
3,10.0,0.0000,0.0,3,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,2.285450e-18,1.397532e-19,6.596853e-19,3.407853e-19,2.860721e-16,9.379412e-18,1.150541e-17,6.190412e-19,0.8384,0.02793
4,10.0,0.0000,0.0,4,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,3.022900e-18,1.848476e-19,8.725471e-19,4.507471e-19,3.783794e-16,1.240588e-17,1.521788e-17,8.187882e-19,0.8384,0.02793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860995,20.0,28.8799,3650.0,30,0.000004,2.085600e-08,2.143210e-09,3.023380e-09,0.000023,3.000150e-20,...,7.052000e-19,4.312235e-20,2.035529e-19,1.051529e-19,8.827059e-17,2.894118e-18,3.550118e-18,1.910118e-19,462.2000,2.61300
860996,20.0,28.8799,3650.0,31,0.000004,2.099090e-08,2.180260e-09,3.056420e-09,0.000023,3.084720e-20,...,1.442650e-18,8.821676e-20,4.164147e-19,2.151147e-19,1.805779e-16,5.920588e-18,7.262588e-18,3.907588e-19,461.4000,2.59600
860997,20.0,28.8799,3650.0,32,0.000004,2.102470e-08,2.163360e-09,3.051300e-09,0.000023,3.056710e-20,...,2.180100e-18,1.333112e-19,6.292765e-19,3.250765e-19,2.728853e-16,8.947059e-18,1.097506e-17,5.905059e-19,461.7000,2.60500
860998,20.0,28.8799,3650.0,33,0.000004,2.090400e-08,2.161430e-09,3.041170e-09,0.000023,3.165860e-20,...,2.917550e-18,1.784056e-19,8.421382e-19,4.350382e-19,3.651926e-16,1.197353e-17,1.468753e-17,7.902529e-19,461.5000,2.60700


In [8]:
df.drop(df.columns[df.columns.str.contains('^(tank_|flow_)', regex=False)], axis=1, inplace=True)

In [5]:
def convert_columns_to_dict(df):
    # Define prefixes to process
    prefixes = ['fuel_']
    exclude_cols = ['fuel_TOT_DH', 'fuel_TOT_GS', 'fuel_TOT_A']
    
    for prefix in prefixes:
        # Get all columns that start with the prefix and aren't in exclude_cols
        cols = [col for col in df.columns if col.startswith(prefix) and col not in exclude_cols]
        
        if not cols:
            continue
            
        # Create the new dictionary column name
        new_col = f"{prefix}value"
        
        # Process row by row instead of using apply
        dict_series = []
        for _, row in df[cols].iterrows():
            row_dict = {
                col[len(prefix):]: val
                for col, val in row.items()
                if val != 0
            }
            dict_series.append(row_dict)
            
        # Assign the new column
        df[new_col] = dict_series
        
        # Drop the original columns
        df = df.drop(columns=cols)
        
    return df

# Usage:
df_converted = convert_columns_to_dict(df)

In [6]:
df_converted

,IE,BU,CT,FMI,fuel_TOT_GS,fuel_TOT_DH,fuel_TOT_A,AN,SF,fuel_value
0,10.0,0.0000,0.0,0,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
1,10.0,0.0000,0.0,1,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
2,10.0,0.0000,0.0,2,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
3,10.0,0.0000,0.0,3,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
4,10.0,0.0000,0.0,4,1.794450e+12,9.92946,1.285680e+13,0.8384,0.02793,"{'F19': 1.19531, 'Na23': 0.396987, 'K39': 0.26..."
...,...,...,...,...,...,...,...,...,...,...
860995,20.0,28.8799,3650.0,30,1.280680e+17,42216.70000,5.408690e+17,462.2000,2.61300,"{'H1': 3.88216e-06, 'H2': 2.0856e-08, 'H3': 2...."
860996,20.0,28.8799,3650.0,31,1.277880e+17,42177.00000,5.401640e+17,461.4000,2.59600,"{'H1': 3.8823e-06, 'H2': 2.09909e-08, 'H3': 2...."
860997,20.0,28.8799,3650.0,32,1.274950e+17,42130.30000,5.395010e+17,461.7000,2.60500,"{'H1': 3.89707e-06, 'H2': 2.10247e-08, 'H3': 2..."
860998,20.0,28.8799,3650.0,33,1.272530e+17,42095.90000,5.387970e+17,461.5000,2.60700,"{'H1': 3.88554e-06, 'H2': 2.0904e-08, 'H3': 2...."


In [7]:
df_converted.to_csv('df_converted_fuel.csv', index=False)

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse the XML file
tree = ET.parse('global/scratch/users/toniooppi/nuclear_data/chain-endf-b8.0.xml')
root = tree.getroot()

# Create a dictionary to store nuclide half-lives
half_lives = {}

# Extract half-lives from XML
for nuclide in root.findall('nuclide'):
    name = nuclide.get('name')
    half_life = nuclide.get('half_life')
    if half_life is not None:
        half_lives[name] = float(half_life)
    else:
        half_lives[name] = 1.0  # Default value for stable nuclides

# Assuming your density dictionary looks like this
densities = {
    'H1': 1.0,
    'H2': 2.0,
    'H3': 3.0,
    'H4': 4.0,
    'H5': 5.0
}

# Convert density dictionary to DataFrame
df = pd.DataFrame.from_dict(densities, orient='index', columns=['density'])

# Add half-life column and multiply
df['half_life'] = df.index.map(half_lives)
df['result'] = df['density'] * df['half_life']

FileNotFoundError: [Errno 2] No such file or directory: 'global/scratch/users/toniooppi/nuclear_data/chain-endf-b8.0.xml'